# sxs_metadata_example.ipynb

This notebook demonstrates how to download and parse the metadata provided by https://data.black-holes.org/catalog.json. This metadata is automatically updated when the catalog data is updated.

First, download the latest version of the ancilliary JSON file.

In [ ]:
import requests
import json
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
request = requests.get("https://data.black-holes.org/catalog.json", headers={'accept': 'application/citeproc+json'})
sxs_full_json = request.json()
for key in sxs_full_json.keys():
    print(key)

## Catalog file description

The 'catalog_file_description' key is a README documenting the contents of the file.

In [ ]:
description = ""
for line in sxs_full_json['catalog_file_description']:
    description += line + "\n"
print(description)

## Simulation metadata

The 'simulations' key contains a dictionary, where each key has the form SXS:TYPE:####, where #### is a 4-digit integer and TYPE is BBH, BNS, or BHNS. This is the same data provided by https://arxiv.org/src/1904.04831/anc/sxs_catalog.json.

In [ ]:
sxs_catalog_json = sxs_full_json['simulations']

These are the available keys for each simulation.

In [ ]:
for key in sorted(sxs_catalog_json['SXS:BBH:0444'].keys()):
    print(key)

Here's an example of processing the metadata: find the N simulations with mass ratio closest to a desired value.

In [ ]:
desired_mass_ratio = 4.444
N = 4

sxs_ids = np.array(sorted(list(sxs_catalog_json.keys())))
mass_ratios_list = []
for sxs_id in sxs_ids:
    if sxs_id.split(":")[-2] == "BBH":
        mass_ratios_list.append(sxs_catalog_json[sxs_id]['reference_mass_ratio'])
mass_ratios = np.array(mass_ratios_list)

mass_ratio_residual_sort = np.argsort(np.abs(mass_ratios - desired_mass_ratio))

closest_N_simulations = sxs_ids[mass_ratio_residual_sort][:N]
closest_N_massratios = mass_ratios[mass_ratio_residual_sort][:N]
print(closest_N_simulations)
print(closest_N_massratios)

Make a plot of mass ratio vs. chi_eff for all available simulations.

In [ ]:
chieff_list = []
for sxs_id in sxs_ids:
    if sxs_id.split(":")[-2] == "BBH":
        chieff_list.append(sxs_catalog_json[sxs_id]['reference_chi_eff'])
chieff = np.array(chieff_list)

In [ ]:
plt.clf()
plt.plot(mass_ratios, chieff, marker='o', linestyle='', markersize=1.0)
plt.xlabel("Mass ratio")
plt.ylabel("$\chi_{eff}$")
plt.show()

## Records metadata

The 'records' key contains a dictionary of metadata for each zenodo record, keyed by record doi.

In [ ]:
doi = sxs_catalog_json['SXS:BBH:0444']['url']

Available keys for each record are the following:

In [ ]:
for key in sxs_full_json['records'][doi].keys():
    print(key)

This example prints a table of available files for a simulation, along with clickable links to download them.

In [ ]:
files = sxs_full_json['records'][doi]['files']
for file in files:
    print(file['filename']+" "+file['links']['download'] + "\n")